# Digit recogition model
dataset from: https://www.kaggle.com/datasets/bhavikjikadara/handwritten-digit-recognition/data

some code and ideas from this video:  https://www.youtube.com/watch?v=tHL5STNJKag 


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
from tqdm.notebook import tqdm

class DigitsDataSetCSV(Dataset):
    def __init__(self, csv_file_path):
        # Load the CSV file into pandas DataFrame
        df = pd.read_csv(csv_file_path)

        # Features are normalized versions of the colour value
        # Targets are the number the images are supposed to represent
        self.features = df.iloc[:, 1:].to_numpy(dtype=np.float32) / 255
        self.targets = df.iloc[:, 0].to_numpy(dtype=np.float32)
        
        # Convert to PyTorch tensors

        self.features = torch.tensor(self.features)
        self.targets = self.targets.astype(int)

        # reshape tensors so timm model can use it.
        reshape_length = self.targets.size
       
        # reshape
        self.features = self.features.reshape([reshape_length, 1 ,28,28])
        new_features = torch.zeros([reshape_length, 3, 28,28])
        
        for i in range(len(self.features)):
            new_features[i] = torch.stack([self.features[i][0],self.features[i][0],self.features[i][0]])
        self.features = new_features
        
    def __len__(self):
        # Return the total number of rows in the dataset
        return len(self.targets)

    def __getitem__(self, idx):
        # Return one sample and its label at the given index
        return self.features[idx], self.targets[idx]

## Getting the data

### Testing if the processing worked

In [2]:
data_dir = './dataset/train/train.csv'
dataset = DigitsDataSetCSV(data_dir)

print(len(dataset))
image, label = dataset[1144]
print(len(dataset[1144]))
print('------')
print(image)
print('shape:')
print(image.shape)
print('label:')
print(label)
print(dataset.features.shape)

40000
2
------
tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
shape:
torch.Size([3, 28, 28])
label:
9
torch.Size([40000, 3, 28, 28])


# Dataloader

In [3]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [4]:
for images, labels in dataloader:
    break

images.shape, labels.shape

(torch.Size([32, 3, 28, 28]), torch.Size([32]))

# Model

Credits to this video https://www.youtube.com/watch?v=tHL5STNJKag for the model code

In [5]:
class DigitClassifier(nn.Module):
    def __init__(self, num_classes=10):
        super(DigitClassifier, self).__init__()
        # Where we define all the parts of the model
        self.base_model = timm.create_model('efficientnet_b0', pretrained=True)
        self.features = nn.Sequential(*list(self.base_model.children())[:-1])

        enet_out_size = 1280 # default of b0
        # Make a classifier
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(enet_out_size, num_classes)
        )
    
    def forward(self, x):
        # Connect these parts and return the output
        x = self.features(x)
        output = self.classifier(x)
        return output

In [6]:
# Create the model
model = DigitClassifier(10)
images.shape # shape should be 32 * 3 * 28 * 28

torch.Size([32, 3, 28, 28])

Example item

In [7]:
example_out = model(images)
print(example_out)
example_out.shape

tensor([[ 0.0836, -0.0166,  0.2783,  0.7731, -0.0599, -0.1051, -0.1037, -0.6930,
          0.3502, -0.5515],
        [ 0.5860,  0.2178,  0.6374,  0.0883, -0.4795, -0.4439,  0.3398, -0.0398,
         -0.3180, -0.1539],
        [ 0.3014,  0.4212,  0.2136,  0.0045,  0.3807, -0.1498,  0.3823,  0.2002,
         -0.1053,  0.5332],
        [-0.0765,  0.1958,  0.0502, -0.8670, -0.4364,  0.4067, -0.6950,  1.1535,
         -0.0396,  0.6129],
        [-0.2373,  0.1309,  0.4254, -0.4438, -0.2037, -0.3147, -0.1878,  0.2273,
         -0.2350,  0.0531],
        [ 0.3122,  0.1521, -0.5201,  0.2693, -0.0440, -0.1594,  0.1197,  0.5458,
         -0.1121,  0.0373],
        [ 0.3098, -0.0643,  0.0709, -0.6889,  0.2100,  0.0095, -0.1500,  0.3689,
         -0.0028, -0.2879],
        [-0.0471,  0.1260, -0.5634, -0.3463, -0.3102,  0.7614,  0.0162, -0.4517,
         -0.2349, -0.2544],
        [ 0.2111, -0.9436, -0.1946, -0.0115, -0.0742,  0.2549, -0.9069, -0.5627,
         -0.6057,  0.3289],
        [-0.4926, -

torch.Size([32, 10])

In [8]:
# Loss function
criterion = nn.CrossEntropyLoss()
# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
print(criterion(example_out, labels))
print(example_out.shape, labels.shape)

tensor(2.3777, grad_fn=<NllLossBackward0>)
torch.Size([32, 10]) torch.Size([32])


Data files

In [10]:
train_folder = './dataset/train/train.csv'
valid_folder = './dataset/valid/valid.csv'
test_folder = './dataset/test/test.csv'

train_dataset = DigitsDataSetCSV(train_folder)
val_dataset = DigitsDataSetCSV(valid_folder)
test_dataset = DigitsDataSetCSV(test_folder)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

Credits to this video https://www.youtube.com/watch?v=tHL5STNJKag for the training loop code.

In [11]:
# Simple training loop
num_epochs = 5
train_losses, val_losses = [], []

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = DigitClassifier(num_classes=10)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc='Training loop'):
        # Move inputs and labels to the device
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * labels.size(0)
    train_loss = running_loss / len(train_loader.dataset)
    train_losses.append(train_loss)
    
    # Validation phase
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc='Validation loop'):
            # Move inputs and labels to the device
            images, labels = images.to(device), labels.to(device)
         
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * labels.size(0)
    val_loss = running_loss / len(val_loader.dataset)
    val_losses.append(val_loss)
    print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {train_loss}, Validation loss: {val_loss}")

Training loop:   0%|          | 0/1250 [00:00<?, ?it/s]

Validation loop:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch 1/5 - Train loss: 0.26317138039376586, Validation loss: 0.08904266508668661


Training loop:   0%|          | 0/1250 [00:00<?, ?it/s]

Validation loop:   0%|          | 0/32 [00:00<?, ?it/s]

...

Validation loop:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch 4/5 - Train loss: 0.06846188057846157, Validation loss: 0.10026531830430031


Training loop:   0%|          | 0/1250 [00:00<?, ?it/s]

Validation loop:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch 5/5 - Train loss: 0.06624809628047515, Validation loss: 0.053615372986067084


In [12]:
plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.legend()
plt.title("Loss over epochs")
plt.show()